<h1><center>Percent Change in Uber Trips After MSG Concert</center></h1>

In [24]:
import numpy as np
import pandas as pd
import pygeohash as g
from datetime import datetime, timedelta, date

pickups_may14=pd.read_csv("uber-raw-data-may14.csv")

In [3]:
# Functions for data cleaning
def convertDateTime(date):
    return datetime.strptime(date, '%m/%d/%Y %H:%M:%S')

def extractWeekday(date):
    return date.weekday()

def extractDate(date):
    return date.date()

def extractHour(date):
    return date.hour

def geoHash(lat,lon,precision):
    return g.encode(lat,lon,precision)

In [4]:
# Apply data cleaning functions

pickups_may14['Date/Time'] = pickups_may14.apply(lambda x: convertDateTime(x['Date/Time']), axis=1)
pickups_may14['Hour'] = pickups_may14.apply(lambda x: extractHour(x['Date/Time']), axis=1)
pickups_may14['Date'] = pickups_may14.apply(lambda x: extractDate(x['Date/Time']), axis=1)
pickups_may14['Weekday'] = pickups_may14.apply(lambda x: extractWeekday(x['Date/Time']), axis=1)
pickups_may14['Geohash7'] = pickups_may14.apply(lambda x: geoHash(x['Lat'],x['Lon'],7), axis=1)

In [14]:
# Merge MSG Geohashes

lookup_msg = pd.read_csv('Lookups/MSG Geohash7 Lookup.csv')
msg_pickups_may14 = pd.merge(pickups_may14,lookup_msg,on='Geohash7', how='inner')

In [65]:
# Define relevant area
pd.options.mode.chained_assignment = None

relevant_area = msg_pickups_may14[(msg_pickups_may14['Weekday']==1) & (msg_pickups_may14['Hour']==22)]
relevant_area['Control'] = relevant_area.apply(lambda x: (x['Date']!=date(2014,5,13))*1, axis=1)


In [83]:
# Calculate average trips in this hour
avg_trips = relevant_area.groupby(['Date','Control']).size().reset_index(name='Trips')
avg_trips_experiment = avg_trips.groupby(['Control'])[['Trips']].mean().reset_index()

In [96]:
# Summarize findings

control = avg_trips_experiment['Trips'][1]
experiment = avg_trips_experiment['Trips'][0]


print("The number of Uber pickups after a concert is {:.2f}.".format(experiment))
print("The number of Uber pickups normally during the same hour is {:.2f}.".format(control))
print("The percent increase in pickups is {:.1f}%.".format((experiment - control)/control*100))

The number of Uber pickups after a concert is 62.00.
The number of Uber pickups normally during the same hour is 5.33.
The percent increase in pickups is 1062.5%.
